In [1]:
import json
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np

/home/suigpt_rag/miniconda3/envs/new_CG/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# 1. import testing Question dataset [Q]
with open("../math_qa_all.json", "r") as qf:
    data = json.load(qf)
    questions: list = [q['Q'] for q in data]
    questions_page: list = [q['page'] for q in data]
    questions_answer: list = [q['A'] for q in data]

# import book by pages [P]
with open("../0906_request.jsonl_output.json", "r") as pf:
    data = json.load(pf)
    pages:list = [data[i]['response']['body']['choices'][0]['message']['content'] for i in range(len(data))]

In [3]:
# 2. import possible models [M]
models_list: list = ['thenlper/gte-large']

# 2.1 model retrieved results [M * Q] = -1
# retrieved_results = [None for i in range(len(questions))]

In [9]:
# 3. For every model in model list
# for model_name in models_list: 
# 3s for loading existing model
model_name = models_list[0]

model = SentenceTransformer(model_name)

In [10]:
# 12.3s
# page_embeddings = []
# 3.1. create embedding for each page, store it in numpy [P * E], E = embedding size
page_embeddings: np.ndarray = model.encode(pages) 
# for page in pages:
#     embeddings = model.encode(page)
#     page_embeddings.append(embeddings)

In [15]:
# 1.7s
# 3.2. for each question in QA dataset, query the question [Q * E]
questions_embeddings: np.ndarray  = model.encode(questions)
# questions_embeddings = []
# for question in questions:
#     embedding = model.encode(question)
#     questions_embeddings.append(embedding)

In [ ]:
# 3.8s
# 3.3. for each question, search the retrieved page rank (index of the numpy 2D array) 
# by calculating the difference between the retrieved page and the true page.
# [Q * K]

# retrieved_page_ranks = []

all_pages = []
for i in range(len(pages)):
    temp_dict: dict = {}
    temp_dict['page_number'] = questions_page[i]
    temp_dict['question'] = questions[i]
    temp_dict['answer'] = questions_answer[i]

for i in range(len(questions)):
    temp_dict: dict = {}
    temp_dict['question_page'] = questions_page[i]
    temp_dict['question'] = questions[i]
    temp_dict['answer'] = questions_answer[i]

# retrieved_page_ranks = []
# for question_embedding in questions_embeddings:
#     question_retrived_page_rank = [float(cos_sim(question_embedding, page_embed)) for page_embed in page_embeddings]
#     sorted_pages = list(np.argsort(question_retrived_page_rank))
#     sorted_pages = [i + 1 for i in sorted_pages]
#     retrieved_page_ranks.append(sorted_pages)
#     break

In [ ]:
# 3.4, for each question, calculate the @1, @3, @10 accuracies. [Q], [Q], [Q]
top1 = 0
top3 = 0
top10 = 0
top20 = 0
for question_index in range(len(retrieved_page_ranks)):
    true_question_page_index = questions_page[question_index]
    retrived_page_rank = retrieved_page_ranks[question_index]

    # print(f"true_question_page_index: {true_question_page_index}")
    # print(f"retrived_page_rank: {retrived_page_rank}")

    # tops
    if true_question_page_index in retrived_page_rank[:1]: top1 += 1
    if true_question_page_index in retrived_page_rank[:3]: top3 += 1
    if true_question_page_index in retrived_page_rank[:10]: top10 += 1
    if true_question_page_index in retrived_page_rank[:20]: top20 += 1

# print(top1)
# print(top3)
# print(top10)
# print(top20)

In [69]:
# instead of test accuracy, test relative distance
top1d = 0
top3d = 0
top10d = 0
top20d = 0

for question_index in range(len(retrieved_page_ranks)):
    true_question_page_index = questions_page[question_index]
    retrived_page_rank = retrieved_page_ranks[question_index]
    top1d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:1])
    top3d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:3])
    top10d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:10])
    top20d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:20])
    

# top1d /= len(retrieved_page_ranks)
# top3d /= len(retrieved_page_ranks)
# top10d /= len(retrieved_page_ranks)
# top20d /= len(retrieved_page_ranks)

In [70]:
# print(top1d)
# print(top3d)
# print(top10d)
# print(top20d)

296.83666666666664
149.93
63.25333333333333
37.05


In [ ]:
for model_name in models_list: 
    model = SentenceTransformer(model_name)
    
    # 3.1. create embedding for each page, store it in numpy [P * E], E = embedding size
    page_embeddings = []
    for page in pages:
        embeddings = model.encode(page)
        page_embeddings.append(embeddings)

    # 3.2. for each question in QA dataset, query the question [Q * E]
    questions_embeddings = []
    for question in questions:
        embedding = model.encode(question)
        questions_embeddings.append(embedding)


    # 3.3. for each question, search the retrieved page rank (index of the numpy 2D array) 
    # by calculating the difference between the retrieved page and the true page.
    # [Q * K]
    retrieved_page_ranks = []
    for question_embedding in questions_embeddings:
        question_retrived_page_rank = [float(cos_sim(question_embedding, pe)) for pe in page_embeddings]
        sorted_pages = list(np.argsort(question_retrived_page_rank))
        sorted_pages = [i + 1 for i in sorted_pages]
        retrieved_page_ranks.append(sorted_pages)
        break

    # 3.4, for each question, calculate the @1, @3, @10 accuracies. [Q], [Q], [Q]
    top1 = 0
    top3 = 0
    top10 = 0
    top20 = 0
    for question_index in range(len(retrieved_page_ranks)):
        true_question_page_index = questions_page[question_index]
        retrived_page_rank = retrieved_page_ranks[question_index]

        # print(f"true_question_page_index: {true_question_page_index}")
        # print(f"retrived_page_rank: {retrived_page_rank}")

        # tops
        if true_question_page_index in retrived_page_rank[:1]: top1 += 1
        if true_question_page_index in retrived_page_rank[:3]: top3 += 1
        if true_question_page_index in retrived_page_rank[:10]: top10 += 1
        if true_question_page_index in retrived_page_rank[:20]: top20 += 1
    
    # 3.4.1 instead of test accuracy, test relative distance
    top1d = 0
    top3d = 0
    top10d = 0
    top20d = 0

    for question_index in range(len(retrieved_page_ranks)):
        true_question_page_index = questions_page[question_index]
        retrived_page_rank = retrieved_page_ranks[question_index]
        top1d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:1])
        top3d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:3])
        top10d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:10])
        top20d += min(abs(i - true_question_page_index) for i in retrived_page_rank[:20])
        

    top1d /= len(retrieved_page_ranks)
    top3d /= len(retrieved_page_ranks)
    top10d /= len(retrieved_page_ranks)
    top20d /= len(retrieved_page_ranks)